# Bibliotecas Usadas

In [81]:
import random
import copy
import math
from tqdm import tqdm_notebook as tqdm
import numpy as np

<h1 style="color:#fff;background-color:#000;padding:30px;margin:0;border:0;text-align:center">JOGO</h1>

Uma classe para simular um jogo de xadrez apenas com rainhas

In [24]:

class Jogo:
    
    """
    Construtor da classe
    
    @param dimensao_tabuleiro: inteiro positivo para dimensao do tabuleiro quadrado
    """
    def __init__(self, dimensao_tabuleiro):
                
        #guardando a dimensao
        self._dimensao = dimensao_tabuleiro
        
        #inicializando o dicionario de pecas
        self._pos_damas = dict()
    
    
    """
    Metodo get da dimensao, disparado quando "self.dimensao" eh executado
    
    @return dimensao: a dimensao do tabuleiro quadrado
    """
    @property
    def dimensao(self): return self._dimensao

    """
    Metodo set da dimensao, disparado quando "self.dimensao = value" eh executado
    por medidas de consistencia, as pecas tem suas posicoes anuladas, porem as quantidades sao as mesmas
    
    @param value: devera ser um inteiro positivo
    """
    @dimensao.setter
    def dimensao(self, value):
         
        #verificacao de consistencia da entrada
        if isinstance(value, int) and value >= 0: 
            self._dimensao = value
            
            #zerando os valores das pecas
            for chave in self._pos_damas: self._pos_damas[chave] = None
            
        else: print("tipo {} nao aceito".format(type(value)))
    
    """
    Metodo get do pos_damas, disparado quando "self.pos_damas" eh executado
    
    @return pos_damas: o dicionario contendo todas as pecas e suas posicoes
    """
    @property
    def pos_damas(self): return self._pos_damas.copy()

    """
    Metodo set do pos_damas, disparado quando "self.pos_damas = value" eh executado
    
    @param value: devera ser um dicionario do tipo: chave --> tuple(x, y) tal que
    tanto x quanto y sejam inteiros dentro do intervalo [0, self.dimensao[
    """
    @pos_damas.setter
    def pos_damas(self, value):
        
        #verificacao de consistencia da entrada
        check = (isinstance(value, dict) and 
                 all(isinstance(e, tuple)
                     and len(e) == 2
                     and all(isinstance(i,int) and
                             i>=0 and i<self.dimensao
                             for i in e)
                     for e in value.values()))
        
        #registrando entrada
        if check: self._pos_damas = value
        
        else: print("{} nao aceito como pos_damas".format(value))
    
    """
    Metodo get para uma peca do tabuleiro, disparado quando "self[idx]" for executado
    
    @return pos_damas[idx]: um tuple(int, int) se a chave existir, None c.c.
    """
    def __getitem__(self, idx): return self._pos_damas.get(idx)

    """
    Metodo set para uma nova peca do tabuleiro ou atualizacao de existente,
    disparado quando "self[idx] = value" for executado
    
    @param idx: novo/existente peca que sera manipulada
    
    @param value: novo valor que sera inserido, devera ser um tuple(x, y) tal que
    tanto x quanto y sejam inteiros dentro do intervalo [0, self.dimensao[
    """
    def __setitem__(self, idx, value):
        
        #verificacao de consistencia
        check = (isinstance(value, tuple)
                 and len(value) == 2 and 
                 all(isinstance(i,int) and
                     i>=0 and
                     i<self.dimensao for i in value)) 
        
        #registrando entrada
        if check:
            self._pos_damas[idx] = value
        else: print("{} nao aceito como valor".format(value))
            
    """
    Metodo para descobrir as jogadas obvias (horizontais e verticais)
    
    @param idx: peca que sera movimentada
    
    @return jogadas: lista de posicoes validas para uma jogada
    """
    def getJogadasObvias(self, idx):
        #obtendo a posicao da peca que sera movimentada
        ponto = self[idx]
        
        #se nao existir, vida que segue
        if not ponto:
            print("peca {} nao inicializada".format(idx))
            return None
        
        #varrendo posicoes nos sentidos
        cima = [(ponto[0], i) for i in range(ponto[1]+1, self.dimensao)]
        
        baixo = [(ponto[0], i) for i in range(ponto[1]-1, -1, -1)]
        
        direita = [(i, ponto[1]) for i in range(ponto[0]+1, self.dimensao)]
        
        esquerda = [(i, ponto[1]) for i in range(ponto[0]-1, -1, -1)]
        
        #concatenando tudo
        return [*cima,
               *baixo,
               *direita,
               *esquerda]
    
    """
    Metodo para descobrir as jogadas diagonais 
    
    @param idx: peca que sera movimentada
    
    @return jogadas: lista de posicoes validas para uma jogada
    """
    def getJogadasDiagonais(self, idx):
        
        #obtendo a posicao da peca que sera movimentada
        ponto = self[idx]
        
        #se nao existir, vida que segue
        if not ponto:
            print("peca {} nao inicializada".format(idx))
            return None

        #varrendo posicoes nos sentidos
        cima_direita = [(ponto[0]+i, ponto[1]+i) for i in range(1, self.dimensao) if ponto[0]+i < self.dimensao and ponto[1]+i < self.dimensao]
        
        baixo_direita = [(ponto[0]+i, ponto[1]-i) for i in range(1, self.dimensao) if ponto[0]+i < self.dimensao and ponto[1]-i >=0 ]
        
        baixo_esquerda = [(ponto[0]-i, ponto[1]-i) for i in range(1, self.dimensao) if ponto[0]-i >= 0 and ponto[1]-i >= 0]
        
        cima_esquerda = [(ponto[0]-i, ponto[1]+i) for i in range(1, self.dimensao) if ponto[0]-i >= 0 and ponto[1]+i < self.dimensao]
        
        #concatenando tudo
        return [*cima_direita,
               *baixo_direita,
               *baixo_esquerda,
               *cima_esquerda]
    
    """
    Metodo para descobrir todas as jogadas possiveis 
    
    @param idx: peca que sera movimentada
    
    @return jogadas: lista de posicoes validas para uma jogada
    """
    def getTodasJogadas(self, idx):
        
        #obtendo a peca que sera movimentada
        ponto = self[idx]
        
        #se nao tiver ponto, vida que segue
        if not ponto:
            print("peca {} nao inicializada".format(idx))
            return None

        #coletando e concatenando as jogadas
        return [*self.getJogadasDiagonais(idx), *self.getJogadasObvias(idx)]
        
    def getEspacosLetais(self):
        
        casas_letais = set()
        for peca in self.pos_damas:
            casas_letais = casas_letais.union(self.getTodasJogadas(peca))
        return list(casas_letais)
        
    def getEspacosLivres(self):
        
        tabuleiro = set( (x,y) for x in range(0,self.dimensao) for y in range(0,self.dimensao) )
        
        casas_letais = set(self.pos_damas.values())
        #casas_letais = set()
        
        for peca in self.pos_damas:
            
            casas_letais = casas_letais.union(self.getTodasJogadas(peca))
        
        return list(tabuleiro - casas_letais)
    
    def __str__(self):
        string = ""
        pos = self.pos_damas.values()
        for i in range(self.dimensao):
            for j in range(self.dimensao):
                if (i, j) in pos:
                    string += "♛"
                else: 
                    string += "□"
                string += " "
            string += "\n"
        return string

        

In [25]:
def tabuleiroInincial(dimensao):
    jogo = Jogo(dimensao)
    jogo.pos_damas = {i: (i,i) for i in range(0, jogo.dimensao)}
    return jogo

In [26]:
def heuristicaOtimizada(jogo):
    
    #peso do tabuleiro
    peso = 0

    #conjunto de pecas nao verificadas
    partes_ocupadas = list(jogo.pos_damas.values())
    
    partes_letais = jogo.getEspacosLetais()
    
    casas_mortas = filter(lambda x: x in partes_letais, partes_ocupadas)
    
    return len(list(casas_mortas))

In [159]:
def heuristicaIcaro(jogo):
    
    #peso do tabuleiro
    peso = 0

    #conjunto de pecas nao verificadas
    partes_ocupadas = list(jogo.pos_damas.keys())
    
    for i in range(len(partes_ocupadas)):
        
        for j in range(i+1, len(partes_ocupadas)):
            
            posi = np.array(jogo[partes_ocupadas[i]])
            
            posj = np.array(jogo[partes_ocupadas[j]])
            
            delta = abs(posi - posj)
            
            if delta[0] == delta[1] or posi[0] == posj[0] or posi[1] == posj[1]:
                peso += 1
        
    return peso    

In [161]:
print(tabuleiroInincial(4))

♛ □ □ □ 
□ ♛ □ □ 
□ □ ♛ □ 
□ □ □ ♛ 



In [162]:
heuristicaIcaro(tabuleiroInincial(4))

6

In [27]:
def heuristicaPadrao(jogo):
    
    #peso do tabuleiro
    peso = 0

    #conjunto de pecas nao verificadas
    pecas_nao_verificadas = list(jogo.pos_damas.keys())

    #enquanto tiver pecas nao verificadas faca:
    for peca in pecas_nao_verificadas:

        #tirar uma peca arbitraria
        #peca = pecas_nao_verificadas.pop()

        #coletar todas as possiveis jogadas dela
        casas_letais = jogo.getTodasJogadas(peca)

        #para as outras pecas
        for chave, valor in jogo.pos_damas.items():

            #verificar se elas estao entre as possiveis jogadas
            if valor in casas_letais:

                #incrementa o peso
                peso += 1

    return peso

In [28]:
def movimentacaoRandomica(jogo):
    
    novo_jogo = copy.deepcopy(jogo)
    
    #peca aleatoria
    peca = random.choice(list(novo_jogo.pos_damas.keys()))
    
    pos = novo_jogo[peca]
    
    #todos os movimentos verticais e horizontais
    movimentos = novo_jogo.getJogadasObvias(peca)
    movimentos_simples_verticais = list(filter(lambda x: x[0] == pos[0], movimentos))
    
    escolha = random.choice(movimentos_simples_verticais)
    
    novo_jogo[peca] = escolha
    
    #novo_jogo.jogada_anterior = (peca, escolha)
    
    return novo_jogo

In [29]:
def simulatedAnnealing(funcao_peso, funcao_jogada, jogo, MAX = 6000, C=3):
    
    atual = jogo
    
    for t in tqdm(range(1, MAX)):
    
        T = C/math.sqrt(t)
        
        #if T == 0: return atual
        
        viz = funcao_jogada(atual)
        
        #print(viz)
        p_atual = funcao_peso(atual)
        p_viz = funcao_peso(viz)
        
        #if T<= 0.0005 or p_viz == 0: return viz
        
        if p_viz < p_atual:
            atual = viz
        
        else:
            probabilidade = math.exp( (p_atual - p_viz)/T )
            #print(probabilidade, funcao_peso(atual))
            atual = random.choices([viz, atual], weights= [probabilidade, 1-probabilidade])[0]
        
    return atual

In [30]:
'''
%%time
teste = simulatedAnnealing(heuristicaPadrao, 
                           movimentacaoRandomica,
                          tabuleiroInincial(40))
'''

'\n%%time\nteste = simulatedAnnealing(heuristicaPadrao, \n                           movimentacaoRandomica,\n                          tabuleiroInincial(40))\n'

In [170]:
'''
%%time
teste = simulatedAnnealing(heuristicaIcaro, 
                           movimentacaoRandomica,
                          tabuleiroInincial(40))
'''

'\n%%time\nteste = simulatedAnnealing(heuristicaIcaro, \n                           movimentacaoRandomica,\n                          tabuleiroInincial(40))\n'

In [169]:
#print(teste)

In [168]:
#heuristicaIcaro(teste)

<p style="color:lime;background-color:#fc0fc0;padding:50px;margin:0;border:0;text-align:center;font-size:40px;font-weight:bold;font-family:URW Chancery L">DEMONSTRAÇÃO</p>

In [34]:
#random.sample(range(1,25),)


In [39]:
def cruzamentoPadrao(pai, mae):
    novo_jogo = copy.deepcopy(pai)
    
    #quantidade_mae = random.choice(range(1,mae.dimensao-1))
    quantidade_mae = round(mae.dimensao/2)
    
    #pecas = random.sample(list(novo_jogo.pos_damas.keys()), quantidade_mae)
    pecas = list(novo_jogo.pos_damas.keys())[quantidade_mae:]
    
    for peca in pecas:
        novo_jogo[peca] = mae[peca]
        
    return novo_jogo

    

In [199]:
def cruzamento2(pai, mae):
    novo_jogo = copy.deepcopy(pai)
    
    #quantidade_mae = random.choice(range(1,mae.dimensao-1))
    quantidade_mae = round(mae.dimensao/2)
    
    #pecas = random.sample(list(novo_jogo.pos_damas.keys()), quantidade_mae)
    pecas = sorted(list(novo_jogo.pos_damas.keys()))[quantidade_mae:]
    
    #print(pecas)
    
    for peca in pecas:
        novo_jogo[peca] = mae[peca]
        
    return novo_jogo

    

In [200]:
def mutacaoPadrao(individuo):
    
    novo_jogo = copy.deepcopy(individuo)
    
    pecas = random.sample(list(novo_jogo.pos_damas.keys()), 2)
    
    pos1 = novo_jogo[pecas[0]]
    
    pos2 = novo_jogo[pecas[1]]
    
    novo_jogo[pecas[0]] = (pos1[0], pos2[1])
    
    novo_jogo[pecas[1]] = (pos2[0], pos1[1])
        
    return novo_jogo

In [201]:
t1 = tabuleiroRandomico(3)
print(t1)

□ □ ♛ 
♛ □ □ 
□ □ ♛ 



In [202]:
t2 = 

SyntaxError: invalid syntax (<ipython-input-202-853baeff4d9a>, line 1)

In [203]:
t = cruzamento2(tabuleiroInincial(4),
                    tabuleiroInincial(4))
print(t)

♛ □ □ □ 
□ ♛ □ □ 
□ □ ♛ □ 
□ □ □ ♛ 



In [204]:
t1 = mutacaoPadrao(t)
print(t1)

□ □ □ ♛ 
□ ♛ □ □ 
□ □ ♛ □ 
♛ □ □ □ 



In [205]:
t2 = cruzamentoPadrao(t, t1)
print(t2)

♛ □ □ □ 
□ ♛ □ □ 
□ □ ♛ □ 
♛ □ □ □ 



In [206]:
def algoritmoGenetico(populacao, f_heuristica, f_cruzamento, f_mutacao, MAX = 5000):
    
    pop = copy.deepcopy(populacao)
    
    for iter in tqdm(range(MAX)):
        
        pesos = np.array(list(map(f_heuristica, pop)))
        
        if 0 in pesos:
            
            index = pesos.index(0)
            
            return pop[index]
        
        #print(pesos)
        
        pais = [random.choices(pop, weights = 1/pesos, k = 2) for x in range(len(pop))]
        
        for index , (pai, mae) in enumerate(pais):
            
            filho = f_cruzamento(pai, mae)
            
            mutacao = random.choices([True, False], weights=[0.01, 0.99])[0]
            
            if mutacao: filho = f_mutacao(filho)
            
            pop[index] = filho
            
    pesos = list(map(lambda x: (f_heuristica(x),x) , pop))
    
    mais_adaptado = sorted(pesos, key = lambda x: x[0])[0][1]
    
    return mais_adaptado
    
            

In [207]:
def tabuleiroRandomico(dimensao):
    jogo = Jogo(dimensao)
    
    l = range(dimensao)
    
    jogo.pos_damas = {i: (i, random.choice(l)) for i in range(dimensao)}
    
    return jogo

In [208]:
pop = [tabuleiroInincial(9) for x in range(100)]

In [209]:
print(pop[0])

♛ □ □ □ □ □ □ □ □ 
□ ♛ □ □ □ □ □ □ □ 
□ □ ♛ □ □ □ □ □ □ 
□ □ □ ♛ □ □ □ □ □ 
□ □ □ □ ♛ □ □ □ □ 
□ □ □ □ □ ♛ □ □ □ 
□ □ □ □ □ □ ♛ □ □ 
□ □ □ □ □ □ □ ♛ □ 
□ □ □ □ □ □ □ □ ♛ 



In [210]:
teste = algoritmoGenetico(pop, heuristicaIcaro, cruzamento2, mutacaoPadrao)

In [211]:
print(teste)

□ □ ♛ □ □ □ □ □ □ 
□ □ □ □ □ □ □ ♛ □ 
□ ♛ □ □ □ □ □ □ □ 
□ □ □ □ ♛ □ □ □ □ 
□ □ □ □ □ □ □ □ ♛ 
□ □ □ □ □ ♛ □ □ □ 
♛ □ □ □ □ □ □ □ □ 
□ □ □ □ ♛ □ □ □ □ 
□ □ □ □ □ □ ♛ □ □ 



In [212]:
heuristicaOtimizada(teste)

2

In [66]:
random.choices(['a','b'], weights=[10,5], k = 2)

['b', 'b']

NameError: name 'np' is not defined

In [114]:
sorted([1,2,3])

[1, 2, 3]

In [165]:
abs(np.array([1,2,3]))

array([1, 2, 3])